In [1]:
import cvxpy as cp
import numpy as np
import numpy.random as npr
import torch
import pandas as pd
import lro
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

# Deterministic Uncertainty Sets

### Formulating the uncertainty set

The first step is to formulate the uncertainty set. There are multiple options, with their associated input parameters

Box uncertainty: $ \{ u \mid\| u \|_\infty \leq \rho \}$ 
- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- center : np.array, optional  
  * An array of dimension n, n the dimension of the uncertanty.
        Translation of the center of the box. If lengths is passed but
        not center it defaults to an array of zeros.
- side : np.array, optional  
  * An array of dimension n, n the dimension of the uncertanty.
        Length of each side of the box. If center is passed but side not
        it defaults to an array of 2.


Ellipsoidal uncertainty: $\{u \mid \| u \|_p \leq \rho \}$ 
- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.

Budget uncertainty: $\{u \mid \| u \|_\infty \leq \rho_1, \| u \|_1 \leq \rho_2\}$ 
- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.


Polyhedral uncertainty: $\{ u \mid Du \leq d\}$
- $D$ : np.array  
- $d$ : np.array  


The uncertainty set can be formulated within the uncertain parameter, which takes in the size of the uncertainty and an uncertainty set as parameters.

In [2]:
# size of uncertain parameter
m = 5

# Box uncertainty set example
box_u = lro.UncertainParameter(m, 
                              uncertainty_set = lro.Box(center=np.zeros(m),
                                                        side=np.ones(m), 
                                                        rho=2.))

# Ellipsoidal uncertainty set example
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = 2, 
                                                                  rho=2.))

# Budget uncertainty set example
budget_u = lro.UncertainParameter(m, 
                                  uncertainty_set = lro.Budget(rho1=2., 
                                                               rho2 = 1.))

# Polyhedral uncertainty set example
poly_u = lro.UncertainParameter(m, 
                                uncertainty_set = lro.Polyhedral(
                                                        D = np.ones((3,m)), 
                                                        d = np.array([1,2,3])))

###  Formulating the Robust Problem

We can now fomulate the Robust Problem, treating the uncertain parameter as a regular parameter.

#### Example 1: Affine transformed LP, ellipsoidal uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu+a)^Tx \leq  10,\\
\end{array}
$$
where $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from the ellipsoidal set above.

In [3]:
#restate the ellipsoidal set
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = 2, 
                                                                  rho=2.))
n = 4

# formulate cvxpy variable
x = cp.Variable(4)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [(P@ellip_u +a)@ x <= 10]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x.value)


LRO objective value:  -2.007530607730419 
LRO x:  [-0.06517727 -0.18582334 -1.12912827 -1.24354975]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + \rho\|P^Tx\|_2 \leq  10,\\
\end{array}
$$

In [4]:
# formulate cvxpy variable
x = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [a@x + 2*cp.norm(P.T@x,2) <= 10]

# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)


Cvxpy objective value:  -2.007530600525304 
Cvxpy x:  [-0.06514715 -0.1858137  -1.12914527 -1.24353695]


We see that they provide the same solution.

#### Example 2: Multiple uncertain terms, box uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu)^Tx  \leq  10 + 2u^Tx,\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$
where again $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from a box uncertainty set. We note that the uncertain parameter can appear more than once in the uncertain constraint.  

The box uncertainty set is indentical to an affine transformed ellipsoidal uncertainty set. If the center of the box is $b$, and side is $s$, formulating the box uncertainaty set is equivalent to formulating an ellipsoidal uncertainty set with $p = \infty$ and an affine transform $\tilde{u} = \mathbf{diag}\{1/2s\}u + b = Au + b$.

With ellipsoidal uncertainty, the above problem is then 
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (PAu+ Pb)^Tx  \leq  10 + (2Au + 2b)^Tx\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$

In [13]:
#formulate using the box set
center = 0.5*np.ones(m)
side = 0.1*np.array([1,2,3,4,5])
box_u = lro.UncertainParameter(m, 
                              uncertainty_set = lro.Box(center=center,
                                                        side=side, 
                                                        rho=2.))
n = 5

# formulate cvxpy variable
x = cp.Variable(n)

# formulate problem constants
P = 3*np.random.rand(n,m)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x)

# formulate constraints
constraints = [P@box_u@ x - 2*box_u@x <= 10 , x >=0, x<=1]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("Box objective value: ", prob_robust.objective.value, "\nBox x: ", x.value)

#formulate using the ellipsoidal set
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = np.inf, 
                                                                  rho=2.))
n = 5

# formulate cvxpy variable
x = cp.Variable(n)

# formulate problem constants
A = np.diag(0.5*side)
b = center

# formulate objective
objective = cp.Minimize(-c@x)

# formulate constraints
constraints = [(P@A@ellip_u + P@b)@ x  <= 10+(2*A@ellip_u+ 2*b)@x, x >=0, x<=1]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("Ellipsoidal objective value: ", prob_robust.objective.value, "\nEllipsoidal x: ", x.value)


Box objective value:  -1.5558083078547698 
Box x:  [1.36960613e-10 8.09996964e-11 1.00000000e+00 1.00000000e+00
 5.83836075e-01]
Ellipsoidal objective value:  -1.5558083078547698 
Ellipsoidal x:  [1.36960613e-10 8.09996964e-11 1.00000000e+00 1.00000000e+00
 5.83836075e-01]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pb)^Tx - 2b^Tx + \rho\|A^TP^Tx - 2A^Tx\|_1 \leq  10,\\
\end{array}
$$

In [14]:
# formulate using cvxpy
x = cp.Variable(5)
# formulate objective
objective = cp.Minimize(-c@x)

# formulate constraints
constraints = [(P@b)@x - 2*b@x + 2*cp.norm((P@A).T@x - 2*A.T@x,p=1)<= 10, x>=0, x<=1]

# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)


Cvxpy objective value:  -1.5558083076253744 
Cvxpy x:  [6.39495526e-09 1.70862932e-09 1.00000000e+00 1.00000000e+00
 5.83836068e-01]


#### Example 3: Multiple uncertain constraints, polyhedral uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (P_1u + a)^Tx  \leq  10\\
& (P_2u)^Tx  \leq  5\\
& x \leq 5,
\end{array}
$$
where $c$, $P_1$, $P_2$, and $a$ are constants, and $u$ is the uncertain parameter from a polyhedral uncertainty set

In [15]:
#restate the ellipsoidal set 
D =  np.vstack((np.eye(m), -2*np.eye(m)))
d = np.concatenate((0.1*np.ones(m), 0.1*np.ones(m)))
poly_u = lro.UncertainParameter(m, 
                                uncertainty_set = lro.Polyhedral(
                                                        D = D, 
                                                        d = d))
n = 4
# formulate cvxpy variable
x = cp.Variable(n)

# formulate problem constants
P1 = 0.5 * np.eye(m)[:n, :]
P2 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x)

# formulate constraints
constraints = [(P1@poly_u +a)@ x <=10, x<=5]
constraints += [(P2@poly_u)@ x <= 5]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x.value)


LRO objective value:  -4.382507115774927 
LRO x:  [ 5.         5.         3.5452386 -8.0850094]


We compare it with its direct reformulation, where each uncertain constraint is treated independently,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + w_1^Td  \leq  10\\
& w_1^TD = P_1^Tx\\
& w_2^Td  \leq  5\\
& w_2^TD = P_2^Tx\\
& w_1, w_2 \geq 0\\
& x \leq 5,
\end{array}
$$
where $w_1$, $w_2$ are new variables introduced.

In [16]:
# formulate using cvxpy
x = cp.Variable(n)
w1 = cp.Variable(2*m)
w2 = cp.Variable(2*m)
# formulate objective
objective = cp.Minimize(-c@x)

# formulate constraints
constraints = [a@x + w1@d <= 10]
constraints += [w1@D == P1.T@x]
constraints += [w2@d <= 5]
constraints += [w2@D == P2.T@x]
constraints += [w1>=0, w2 >= 0, x <=5]
# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)

Cvxpy objective value:  -4.382507115791347 
Cvxpy x:  [ 5.         5.         3.5452386 -8.0850094]


#### Example 4: Direct comparison constraints, budget uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu + a)^Tx  \leq  10\\
& x \geq P_1u,
\end{array}
$$
where $c$, $P$,and $a$ are constants, and $u$ is the uncertain parameter from a budet uncertainty set. 
We can include constraints that relates directly the values of $x$ and $u$, as in the second constraint.

In [17]:
#restate the ellipsoidal set
budget_u = lro.UncertainParameter(m, 
                                  uncertainty_set = lro.Budget(rho1=2., 
                                                               rho2 = 1.))
n = 5
# formulate cvxpy variable
x = cp.Variable(n)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
P1 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [(P@budget_u +a)@ x <= 10]
constraints += [x >= P1@budget_u]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x.value)


LRO objective value:  10.805361519926496 
LRO x:  [2.64048752 2.77754981 2.86105771 2.67588584 2.74835411]


We compare this with its direct reformulation,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + \rho_1\|y\|_1 + \rho_2\|P^Tx - y\|_\infty  \leq  10\\
& -x_i + \rho_1\|z_i\|_1 + \rho_2\|(P_1^T)_i - z_i\|_\infty \leq 0, \quad i = 1,\dots,n
\end{array}
$$
where $(P_1^T)_i$ is the $i$'th row of $P_1$.

In [18]:
# formulate using cvxpy
x = cp.Variable(n)
y = cp.Variable(m)
z = cp.Variable((n,m))

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [a@x + 2*cp.norm(y,1) + cp.norm(P.T@x-y,np.inf) <= 10]
for i in range(n):
  constraints += [-x[i] + 2*cp.norm(z[i],1) + cp.norm(P1.T@(np.eye(m)[i]) - z[i],np.inf) <= 0]
# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)

Cvxpy objective value:  10.805361527616576 
Cvxpy x:  [2.64048752 2.77754981 2.86105771 2.67588584 2.74835411]


In [17]:
df

,step,Opt_val,Eval_val,Loss_val,A_norm
0,0,-3.499268,-3.102079,-3.138209,1.0
1,1,-4.402934,-3.428694,-3.517094,1.2205309
2,2,-5.631884,-3.683755,-3.774968,1.5240312
3,3,-6.806759,-3.759098,-3.861156,1.8163142
4,4,-7.796053,-3.740569,-3.882948,2.0636451
...,...,...,...,...,...
95,95,-13.5421,-3.412642,-3.923714,3.504712
96,96,-13.547321,-3.412688,-3.923717,3.506023
97,97,-13.552546,-3.412733,-3.923721,3.5073345
98,98,-13.557773,-3.412779,-3.923724,3.5086467
